<a href="https://colab.research.google.com/github/amin-jalilzadeh-tu/JavaScript-from-Beginner-to-Professional/blob/main/GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install geemap

import ee
import geemap.eefolium as geemap
from google.colab import auth



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.8 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'geemap.eefolium'

In [1]:
import ee
from google.colab import auth

In [4]:
# Authenticate and initialize the Earth Engine session
auth.authenticate_user()
ee.Initialize(project='qgis-tu')


# https://console.cloud.google.com/apis/api/earthengine.googleapis.com/metrics?project=qgis-tu&authuser=4&supportedpurview=project


In [6]:
# Define the area of interest (AOI) with coordinates around Delft
aoi = ee.Geometry.Polygon([
    [4.2986, 52.0236],
    [4.2986, 51.9609],
    [4.4207, 51.9609],
    [4.4207, 52.0236]
])
print("AOI Defined: ", aoi.getInfo())


AOI Defined:  {'type': 'Polygon', 'coordinates': [[[4.2986, 51.9609], [4.4207, 51.9609], [4.4207, 52.0236], [4.2986, 52.0236], [4.2986, 51.9609]]]}


In [7]:
# Define the time range
start_date = '2023-06-01'
end_date = '2023-06-30'

# Load Sentinel-2 L2A imagery
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Print the size of the ImageCollection
print("Number of images in Collection: ", s2.size().getInfo())


Number of images in Collection:  2


In [8]:
# Select bands
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

# Clip the image to the AOI and select the bands
clipped = s2.median().clip(aoi).select(bands)
print("Clipped and Selected Bands: ", clipped.getInfo())


Clipped and Selected Bands:  {'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [3, 51], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [3, 51], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [3, 51], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [3, 51], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 2], 'origin': [3, 51], 'crs': 'EPSG:4326', 'crs_t

In [15]:
import folium

# Function to add Earth Engine layers to Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}

# Create a folium map object.
my_map = folium.Map(location=[51.9921, 4.3546], zoom_start=12, height=500)

# Add the imagery to the map object.
my_map.add_ee_layer(clipped, vis_params, 'Sentinel-2 RGB')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)


In [16]:
# Number of clusters
num_clusters = 5

# Instantiate the clusterer and set parameters
clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(
    features=clipped.sample(**{
        'region': aoi,
        'scale': 30,
        'numPixels': 5000  # Adjust based on your area and needs
    })
)

# Cluster the input using the trained clusterer
result = clipped.cluster(clusterer)

# Print cluster information
print("Clustering result: ", result.getInfo())


Clustering result:  {'type': 'Image', 'bands': [{'id': 'cluster', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [2, 2], 'origin': [3, 51], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[4.2986, 51.9609], [4.4207, 51.9609], [4.4207, 52.0236], [4.2986, 52.0236], [4.2986, 51.9609]]]}}}


In [17]:
# Set visualization parameters for clustering
cluster_vis_params = {
    'min': 0,
    'max': num_clusters - 1,
    'palette': ['red', 'green', 'blue', 'yellow', 'purple']  # Add more colors if more clusters
}

# Create a folium map object for the clustering result
cluster_map = folium.Map(location=[51.9921, 4.3546], zoom_start=12, height=500)

# Add the clustering result to the map object
cluster_map.add_ee_layer(result, cluster_vis_params, 'Clustered Image')

# Add a layer control panel to the map
cluster_map.add_child(folium.LayerControl())

# Display the map with the clustering result
display(cluster_map)
